## C FFI

In [ ]:
import ctypes
from ctypes import POINTER, c_double, c_size_t

clibrary = ctypes.CDLL("cpython.so")

clibrary.set_frames_py.argtypes = [
    POINTER(c_double),  # table_x
    POINTER(c_double),  # table_y
    POINTER(c_double),  # table_z
    POINTER(c_double),  # slider_x
    POINTER(c_double),  # slider_y
    POINTER(c_double),  # slider_z
    POINTER(c_double),  # state_A
    POINTER(c_double),  # state_B
    c_size_t            # num_of_frames
]
clibrary.set_frames_py.restype = ctypes.c_bool

## Visualisation

In [ ]:
import plotly.graph_objects as go
import numpy as np
from numpy import cos, sin
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output
from dataclasses import dataclass

In [ ]:
AXIS_RANGE = [-20, 20] 
MESH_INDICES = [(0, 1, 6), (2, 3, 6), (4, 5, 6), (1, 4, 6), (3, 0, 6), (5, 2, 6)]
STEP_SIZE = 0.0000001
TOTAL_FRAMES = 100

x_widget_A = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='X, state A')
y_widget_A = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Y, state A')
z_widget_A = widgets.FloatSlider(value=0, min=0, max=20, step=STEP_SIZE, description='Z, state A')
phi_widget_A = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='φ, state A')
theta_widget_A = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='θ, state A')
psi_widget_A = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='ψ, state A')

x_widget_B = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='X,state B')
y_widget_B = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Y, state B')
z_widget_B = widgets.FloatSlider(value=0, min=0, max=20, step=STEP_SIZE, description='Z, state B')
phi_widget_B = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='φ, state B')
theta_widget_B = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='θ, state B')
psi_widget_B = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='ψ, state B')

frame_widget = widgets.IntSlider(value=0, min=0, max=TOTAL_FRAMES-1, step=1, description='Frame')
success_widget = widgets.Valid(value=False, description='Success?')

WIDGETS_A = [x_widget_A, y_widget_A, z_widget_A, phi_widget_A, theta_widget_A, psi_widget_A]
WIDGETS_B = [x_widget_B, y_widget_B, z_widget_B, phi_widget_B, theta_widget_B, psi_widget_B]

In [ ]:
x_widget_A.value = 0.0
y_widget_A.value = 0.0
z_widget_A.value = 10.81
phi_widget_A.value = 0.0
theta_widget_A.value = 0.0
psi_widget_A.value = 0.0
x_widget_B.value = 0.0

x_widget_B.value = 0.0
y_widget_B.value = -1.45
z_widget_B.value = 3.0
phi_widget_B.value = -0.75
theta_widget_B.value = 0.0
psi_widget_B.value = 0.0
frame_widget.value = TOTAL_FRAMES - 1;

In [ ]:
def initialize_plot():
    fig = go.FigureWidget()
    fig.update_layout(
        scene=dict(xaxis=dict(range=AXIS_RANGE), yaxis=dict(range=AXIS_RANGE), zaxis=dict(range=AXIS_RANGE)),
        title='6-DOF robot with vertical parallel rails',
         width=800, height=400
    )
    
    fig.add_scatter3d(mode='markers', marker=dict(size=3), line=dict(color='blue'), name='State A Points')
    fig.add_scatter3d(mode='markers', marker=dict(size=3), line=dict(color='red'),  name='State T Points')
    fig.add_mesh3d(color='blue', opacity=0.5, name='State A Mesh')
    fig.add_mesh3d(color='red', opacity=0.5,  name='State T Mesh')
    fig.add_scatter3d(mode='lines+markers', marker=dict(size=2, symbol='x', color='blue'), line=dict(color='blue', width=2), name='State A Arms')
    fig.add_scatter3d(mode='lines+markers', marker=dict(size=2, symbol='x', color='red'), line=dict(color='red', width=2), name='State T Arms')
    fig.add_scatter3d(mode='lines', line=dict(color='blue', width=2), showlegend=False)
    fig.add_scatter3d(mode='lines', line=dict(color='red', width=2), showlegend=False)
    
    initial_sliders = get_base()
    vertical_lines_x = []
    vertical_lines_y = []
    vertical_lines_z = []
    for slider in initial_sliders:
        vertical_lines_x.extend([slider[0], slider[0], None])
        vertical_lines_y.extend([slider[1], slider[1], None])
        vertical_lines_z.extend([0, 20, None]) 
        
    fig.add_scatter3d(
        x=vertical_lines_x,
        y=vertical_lines_y,
        z=vertical_lines_z,
        mode='lines',
        line=dict(color='black', width=2),
        name='Rails'
    )

    fig.update_layout(scene=dict(xaxis=dict(range=AXIS_RANGE, title='X Axis'), yaxis=dict(range=AXIS_RANGE, title='Y Axis'), zaxis=dict(range=AXIS_RANGE, title='Z Axis'), aspectmode='cube'), margin=dict(l=0, r=0, b=0, t=0))
    return fig

def get_base():
    state_A = (ctypes.c_double * 6)(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
    state_B = (ctypes.c_double * 6)(0.0, 0.0, 0.0, 0.0, 0.0, 0.0)

    table_x = (ctypes.c_double * (TOTAL_FRAMES * 8))()
    table_y = (ctypes.c_double * (TOTAL_FRAMES * 8))()
    table_z = (ctypes.c_double * (TOTAL_FRAMES * 8))()
    slider_x = (ctypes.c_double * (TOTAL_FRAMES * 6))()
    slider_y = (ctypes.c_double * (TOTAL_FRAMES * 6))()
    slider_z = (ctypes.c_double * (TOTAL_FRAMES * 6))()

    success = clibrary.set_frames_py(
        table_x, table_y, table_z,
        slider_x, slider_y, slider_z,
        state_A, state_B,
        TOTAL_FRAMES
    )
    
    initial_sliders = np.array([[slider_x[i], slider_y[i], slider_z[i]] for i in range(6)])

    return initial_sliders 
    

def update_plot(change):
    state_A = (ctypes.c_double * 6)(x_widget_A.value, y_widget_A.value, z_widget_A.value, phi_widget_A.value, theta_widget_A.value, psi_widget_A.value)
    state_B = (ctypes.c_double * 6)(x_widget_B.value, y_widget_B.value, z_widget_B.value, phi_widget_B.value, theta_widget_B.value, psi_widget_B.value)

    table_x = (ctypes.c_double * (TOTAL_FRAMES * 8))()
    table_y = (ctypes.c_double * (TOTAL_FRAMES * 8))()
    table_z = (ctypes.c_double * (TOTAL_FRAMES * 8))()
    slider_x = (ctypes.c_double * (TOTAL_FRAMES * 6))()
    slider_y = (ctypes.c_double * (TOTAL_FRAMES * 6))()
    slider_z = (ctypes.c_double * (TOTAL_FRAMES * 6))()

    success = clibrary.set_frames_py(
        table_x, table_y, table_z,
        slider_x, slider_y, slider_z,
        state_A, state_B,
        TOTAL_FRAMES
    )

    if success:
        initial_frame_points = np.array([[table_x[i], table_y[i], table_z[i]] for i in range(8)])
        selected_frame_points = np.array([[table_x[8*frame_widget.value + i], table_y[8*frame_widget.value + i], table_z[8*frame_widget.value + i]] for i in range(8)])

               
        initial_sliders = np.array([[slider_x[i], slider_y[i], slider_z[i]] for i in range(6)])
        selected_sliders = np.array([[slider_x[6*frame_widget.value + i], slider_y[6*frame_widget.value + i], slider_z[6*frame_widget.value + i]] for i in range(6)])
        
        success_widget.value = True;
        with plot.batch_update():
            plot.data[0].x, plot.data[0].y, plot.data[0].z = initial_frame_points[:, 0], initial_frame_points[:, 1], initial_frame_points[:, 2]
            plot.data[1].x, plot.data[1].y, plot.data[1].z = selected_frame_points[:, 0], selected_frame_points[:, 1], selected_frame_points[:, 2]

            plot.data[2].x, plot.data[2].y, plot.data[2].z = initial_frame_points[:, 0][0:6], initial_frame_points[:, 1][0:6], initial_frame_points[:, 2][0:6]
            plot.data[3].x, plot.data[3].y, plot.data[3].z = selected_frame_points[:, 0][0:6], selected_frame_points[:, 1][0:6], selected_frame_points[:, 2][0:6]

            initial_arms_x = []
            initial_arms_y = []
            initial_arms_z = []
            
            selected_arms_x = []
            selected_arms_y = []
            selected_arms_z = []
            
            for i in range(6):
                initial_arms_x.extend([initial_sliders[i, 0], initial_frame_points[i, 0], None])
                initial_arms_y.extend([initial_sliders[i, 1], initial_frame_points[i, 1], None])
                initial_arms_z.extend([initial_sliders[i, 2], initial_frame_points[i, 2], None])

                selected_arms_x.extend([selected_sliders[i, 0], selected_frame_points[i, 0], None])
                selected_arms_y.extend([selected_sliders[i, 1], selected_frame_points[i, 1], None])
                selected_arms_z.extend([selected_sliders[i, 2], selected_frame_points[i, 2], None])

            plot.data[4].x, plot.data[4].y, plot.data[4].z = initial_arms_x, initial_arms_y, initial_arms_z
            plot.data[5].x, plot.data[5].y, plot.data[5].z = selected_arms_x, selected_arms_y, selected_arms_z

            plot.data[6].x, plot.data[6].y, plot.data[6].z = [initial_frame_points[6, 0], initial_frame_points[7, 0]], [initial_frame_points[6, 1], initial_frame_points[7, 1]], [initial_frame_points[6, 2], initial_frame_points[7, 2]]
            plot.data[7].x, plot.data[7].y, plot.data[7].z = [selected_frame_points[6, 0], selected_frame_points[7, 0]], [selected_frame_points[6, 1], selected_frame_points[7, 1]], [selected_frame_points[6, 2], selected_frame_points[7, 2]]

    else:
        success_widget.value = False;

plot = initialize_plot()

for widget in WIDGETS_A + WIDGETS_B + [frame_widget]:
    widget.observe(update_plot, names='value')

update_plot(None)

In [ ]:
display(HBox([VBox(WIDGETS_A), VBox(WIDGETS_B), VBox([frame_widget, success_widget])]))
display(plot)